In [42]:
import pandas as pd
import numpy as np
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import NMF, accuracy
from surprise.model_selection import GridSearchCV,RandomizedSearchCV
from scipy.stats import randint as sp_randint



In [43]:
#reading data in dataframe
ratings=pd.read_csv('MFtrain.csv')
ratings.columns=['UserID','MovieID','Rating','Timestamp','class']
ratings

,UserID,MovieID,Rating,Timestamp,class
0,1,1193,5,978300760,N
1,1,2294,4,978824291,N
2,1,3186,4,978300019,N
3,1,1566,4,978824330,N
4,1,588,4,978824268,N
...,...,...,...,...,...
1000204,6040,1594,3,964828599,N
1000205,6040,1587,1,956716374,N
1000206,6040,3182,5,984195682,N
1000207,6040,300,2,956704716,N


In [5]:
m=ratings.groupby(['UserID','class']).size()
m=m.reset_index()
m['MovieID']=0
m['Rating']=100
m.columns=('UserID', 'clas','0' , 'MovieID', 'Rating')
m.loc[m.clas=='Y','Rating']=1  

In [112]:
m = m.drop(["clas", "0"], axis=1)
m

,UserID,MovieID,Rating
0,1,0,100
1,2,0,100
2,3,0,100
3,4,0,100
4,5,0,100
...,...,...,...
6035,6036,0,1
6036,6037,0,100
6037,6038,0,100
6038,6039,0,100


In [113]:
x=ratings.drop(["class", "Timestamp","date","year"], axis=1)
x

,UserID,MovieID,Rating
136767,1,1193,5
136797,1,2294,4
136798,1,3186,4
136799,1,1566,4
136800,1,588,4
...,...,...,...
999977,6040,1594,3
999976,6040,1587,1
999975,6040,3182,5
999983,6040,300,2


In [131]:
frames=[x,m]
res=pd.concat(frames,sort=False)
res
res.sort_values(['UserID', 'MovieID'], ascending=[True, True])


,UserID,MovieID,Rating
0,1,0,100
136807,1,1,5
136792,1,48,5
136806,1,150,5
136811,1,260,4
...,...,...,...
1000120,6040,3683,4
1000178,6040,3703,4
1000183,6040,3735,4
1000191,6040,3751,4


In [99]:
m.Rating==0

0       True
1       True
2       True
3       True
4       True
        ... 
6035    True
6036    True
6037    True
6038    True
6039    True
Name: Rating, Length: 6040, dtype: bool

In [44]:
#reading df into surprise dataset
reader = Reader()
data = Dataset.load_from_df(ratings[['UserID', 'MovieID', 'Rating']], reader)
data

In [41]:
param_grid = {'n_factors':[10,40],'n_epochs': [10,40]}
gs = GridSearchCV(NMF, param_grid, measures=['rmse'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])


0.9572564433910644
{'n_factors': 10, 'n_epochs': 40}


In [45]:
#train test split
train,test = train_test_split(data, test_size=0.25)
##trainset1 = data.build_full_trainset()


In [49]:
#matrix factorization using NMF
algo = NMF(n_factors=15,n_epochs=20)
algo.fit(train)

In [52]:
l=(np.dot(algo.pu,(algo.qi).T))
(pd.DataFrame(l)).iloc[:,148]

0       3.572535
1       3.931676
2       3.940619
3       2.917939
4       2.807479
          ...   
6035    4.083214
6036    4.359333
6037    3.491756
6038    3.451437
6039    4.042977
Name: 148, Length: 6040, dtype: float64

In [53]:
Latent=pd.DataFrame(algo.pu)
Latent

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.884311,0.443311,0.868482,0.394021,0.581457,0.712291,0.342407,0.390810,0.654841,0.323165,0.320215,0.595408,0.130642,0.664447,0.257416
1,0.471612,0.363446,0.528385,0.192473,0.665772,0.874619,0.078209,0.817373,0.514178,0.858726,0.278950,0.557729,0.716003,0.937690,0.545259
2,0.505348,0.692231,0.558572,0.109994,0.430222,0.888176,0.597502,0.781722,0.843986,0.910651,0.778865,0.601963,0.093044,0.736050,0.615220
3,0.516067,0.698254,0.242585,0.582713,0.662511,0.536855,0.449184,0.024663,0.289786,0.266143,0.436951,0.148873,0.485240,0.162118,0.618431
4,0.422481,0.556628,0.322496,0.316669,0.299352,0.514384,0.597998,0.533540,0.270785,0.309643,0.542064,0.031257,0.401506,0.641673,0.459233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.677103,0.257722,0.172697,0.323115,1.809355,0.634862,0.383474,0.169636,0.034119,0.458609,0.151559,0.995208,0.496476,0.607790,0.940619
6036,0.907002,0.896166,0.174010,1.099503,0.758999,1.000565,0.327440,0.039271,0.599238,0.105824,0.259919,0.899297,0.546385,0.369713,0.705136
6037,0.511018,0.433055,0.729607,0.455667,0.657286,0.866305,0.559641,0.650600,0.399287,0.159538,0.865033,0.087867,0.098725,0.253039,0.649382
6038,1.417711,0.822576,0.302038,0.129377,0.590776,0.465177,0.411006,0.032290,0.874581,0.004255,1.195492,0.438173,0.064148,0.449108,0.299706


In [54]:
x=ratings.groupby(['UserID','class']).size()
newx=x.reset_index()

In [55]:
Latent['UserID']=newx.UserID
Latent['Class']=newx.iloc[:,1]
Latent

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,UserID,Class
0,0.884311,0.443311,0.868482,0.394021,0.581457,0.712291,0.342407,0.390810,0.654841,0.323165,0.320215,0.595408,0.130642,0.664447,0.257416,1,N
1,0.471612,0.363446,0.528385,0.192473,0.665772,0.874619,0.078209,0.817373,0.514178,0.858726,0.278950,0.557729,0.716003,0.937690,0.545259,2,N
2,0.505348,0.692231,0.558572,0.109994,0.430222,0.888176,0.597502,0.781722,0.843986,0.910651,0.778865,0.601963,0.093044,0.736050,0.615220,3,N
3,0.516067,0.698254,0.242585,0.582713,0.662511,0.536855,0.449184,0.024663,0.289786,0.266143,0.436951,0.148873,0.485240,0.162118,0.618431,4,N
4,0.422481,0.556628,0.322496,0.316669,0.299352,0.514384,0.597998,0.533540,0.270785,0.309643,0.542064,0.031257,0.401506,0.641673,0.459233,5,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.677103,0.257722,0.172697,0.323115,1.809355,0.634862,0.383474,0.169636,0.034119,0.458609,0.151559,0.995208,0.496476,0.607790,0.940619,6036,Y
6036,0.907002,0.896166,0.174010,1.099503,0.758999,1.000565,0.327440,0.039271,0.599238,0.105824,0.259919,0.899297,0.546385,0.369713,0.705136,6037,N
6037,0.511018,0.433055,0.729607,0.455667,0.657286,0.866305,0.559641,0.650600,0.399287,0.159538,0.865033,0.087867,0.098725,0.253039,0.649382,6038,N
6038,1.417711,0.822576,0.302038,0.129377,0.590776,0.465177,0.411006,0.032290,0.874581,0.004255,1.195492,0.438173,0.064148,0.449108,0.299706,6039,N


In [56]:
Latent.to_csv('Latent.csv',index=False)

In [157]:
'


' in predict

SyntaxError: invalid syntax (<ipython-input-157-62644b30ed0c>, line 1)

In [51]:
#accuracy calculation
accuracy.rmse(predict)

RMSE: 0.9323


0.9323467441372859